In [ ]:
#r "nuget: Deedle, 2.3.0"
#r "nuget: Plotly.NET, 2.0.0-beta9"
#r "nuget: Plotly.NET.Interactive, 2.0.0-beta9"
#r "nuget: FSharp.Stats, 0.4.1"

#i "nuget:https://www.myget.org/F/gregs-experimental-packages/api/v3/index.json"
#r "nuget:Deedle.DotNet.Interactive.Extension, 0.1.0-alpha6"

In [ ]:
open Deedle
open Plotly.NET
open System
open System.Net.Http
open System.Net.Http.Json
open System.Text.Json
open System.Text.Json.Serialization

In [ ]:
let data =
    Frame.ReadCsv("../data/at_load_hourly_mw.csv", hasHeaders = true, culture = "en-US", inferTypes = true, inferRows = 5_000)
    |> Frame.indexRowsDate "TimeStamp"

data

In [ ]:
let baseUrl = "http://localhost:5000"

let client = new HttpClient()
client.BaseAddress <- Uri(baseUrl)

client.GetAsync("/")
|> Async.AwaitTask
|> Async.RunSynchronously

In [ ]:
type LinearForecastResponse =
    { LoadForecast: float }

In [ ]:
let from = DateTime.Parse("2019-03-10T00:00:00")
let horizon = 24 * 10

let response =
    client.PostAsJsonAsync("/linear-forecast", {| From = from; Horizon = horizon |})
    |> Async.AwaitTask
    |> Async.RunSynchronously

let forecastedValues =
    response.Content.ReadFromJsonAsync<LinearForecastResponse array>()
    |> Async.AwaitTask
    |> Async.RunSynchronously
    |> Array.map (fun lf -> lf.LoadForecast)

forecastedValues
|> Array.take 10

In [ ]:
let keys, actualValues =
    data?Value
    |> Series.observations
    |> Seq.skipWhile (fun (k, _) -> k <> from)
    |> Seq.take horizon
    |> Seq.unzip

let predChart =
    Seq.zip keys forecastedValues
    |> fun xy -> Chart.Line(xy, UseWebGL = true, Name = "Predicted")

let actualChart =
    Seq.zip keys actualValues
    |> fun xy -> Chart.Line(xy, UseWebGL = true, Name = "Actual")

[ actualChart; predChart ]
|> Chart.Combine

In [ ]:
type AlternativeLoadForecast =
  { Forecast: float32 array
    LowerBound: float32 array
    UpperBound: float32 array }

In [ ]:
let response =
    client.PostAsJsonAsync("/ssa-forecast", {| From = from; Horizon = horizon |})
    |> Async.AwaitTask
    |> Async.RunSynchronously

let forecastedValues =
    response.Content.ReadFromJsonAsync<AlternativeLoadForecast>()
    |> Async.AwaitTask
    |> Async.RunSynchronously
    |> fun forecast -> forecast.Forecast

forecastedValues
|> Array.take 10

In [ ]:
let predChart =
    Seq.zip keys forecastedValues
    |> fun xy -> Chart.Line(xy, UseWebGL = true, Name = "Predicted")

let actualChart =
    Seq.zip keys actualValues
    |> fun xy -> Chart.Line(xy, UseWebGL = true, Name = "Actual")

[ actualChart; predChart ]
|> Chart.Combine